# Demo notebook for Two Model Readers

In [ ]:
import kamodo_ccmc.flythrough.model_wrapper as MW
MW.Choose_Model('')  # Show list of models currently available

In [ ]:
# Retrieve model reader and access model data for the GITM model.
import kamodo_ccmc.flythrough.model_wrapper as MW
reader = MW.Model_Reader('CTIPe')
file_dir = 'C:/Users/rringuet/Kamodo_Data/CTIPe/Storm_201303/'
kamodo_object_ctipe = reader(file_dir+'2013-03-16', variables_requested=['T_i'])
kamodo_object_ctipe

In [ ]:
# Retrieve model reader and access model data for the GITM model.
# Note the identical syntax!
import kamodo_ccmc.flythrough.model_wrapper as MW
reader = MW.Model_Reader('GITM')
file_dir = 'C:/Users/rringuet/Kamodo_Data/GITM/Storm_201303/'
kamodo_object_gitm = reader(file_dir+'*130316', variables_requested=['T_i'])
kamodo_object_gitm

In [ ]:
# Plot both at chosen time and height
from kamodo import Kamodo
kamodo_object = Kamodo()
kamodo_object['CTIPeT_i[K]'] = kamodo_object_ctipe['T_i_ijk']
kamodo_object['GITMT_i[K]'] = kamodo_object_gitm['T_i_ijk']
kamodo_object

In [ ]:
fig = kamodo_object.plot('CTIPeT_i', plot_partial={'CTIPeT_i': {'time': 12., 'height': 500.}})
fig.update_traces(colorscale="Viridis", ncontours=200, 
                  contours=dict(coloring="fill",showlines=False))
fig

In [ ]:
fig = kamodo_object.plot('GITMT_i', plot_partial={'GITMT_i': {'time': 12., 'height': 500.}})
fig.update_traces(colorscale="Viridis", ncontours=200, 
                  contours=dict(coloring="fill",showlines=False))
fig

In [ ]:
# Demostrate syntax to access the regular interpolator
# Best for single point calculations
kamodo_object_ctipe.T_i([12, -20, 20, 500])

In [ ]:
# Demostrate syntax to access the gridded interpolator
# Best for slice calculations
kamodo_object_ctipe.T_i_ijk(time=12, lon=-20, lat=20, height=500)